In [1]:
pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 18.9 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp38-cp38-linux_x86_64.whl size=236302 sha256=190e9285aac60f7291fbe380878b5c5f45d6c0ac9879b5e4c8b7ab167306b353
  Stored in directory: /root/.cache/pip/wheels/25/d4/61/dbd8edd1a0d656be7b4267c85db3b61951eb60016a0154a122
Successfully built pickle5


In [41]:
from pathlib import Path
from google.colab import files
import pandas as pd
import random
import numpy as np
from tqdm.notebook import tqdm 
import torch
from torch import tensor, nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pickle5 as pickle

In [3]:
# Data processed in seperate notebook. Dataset contains every move of thousands
# of games from start to finish. A particular position is represented by a 
# 12x8x8 bitboard.

with open("/content/drive/MyDrive/chess/processed_data",'rb') as pickled_file:
  X=pickle.load(pickled_file)

In [25]:
# Creating the metric the neural network will try to predict

# the score value is bounded from [-1,1] 
# score is positive when blacks wins that game and negative when white wins
# the closer the score is to -1 or 1, the closer the game is to a checkmate

# We also ony include games that were rated and the minimum elo rating of the 
# players was above 1300

X2=X[(X['black_rating']>1300) & (X['white_rating']>1300) & X['rated']]
X2.loc[:,'turns_until_win']=X2['turns']-X2['turn']
winner=(X2['winner']=='black').apply(lambda x: 1 if x else -1)
X2.loc[:,'score']=X2['turns_until_win'].apply(lambda x:1/(x+1) )
X2.loc[:,'score']=X2['score']*winner

In [40]:
X2

,moves,rated,white_rating,black_rating,turns,winner,turn,turns_until_win,score
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1496,1500,61,white,1,60,-0.016393
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1496,1500,61,white,2,59,-0.016667
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1496,1500,61,white,3,58,-0.016949
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1496,1500,61,white,4,57,-0.017241
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1496,1500,61,white,5,56,-0.017544
...,...,...,...,...,...,...,...,...,...
6311,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1878,1720,115,white,111,4,-0.200000
6311,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1878,1720,115,white,112,3,-0.250000
6311,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1878,1720,115,white,113,2,-0.333333
6311,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",True,1878,1720,115,white,114,1,-0.500000


In [ ]:
# creating data loader
from sklearn.model_selection import train_test_split

batch_size=4096

class ChessDataSet(Dataset):
  def __init__(self,df):
    self.data=np.array(df[['moves','score']])
  def __len__(self):
    return len(self.data)
  def __getitem__(self,idx):
    return torch.as_tensor(self.data[idx,0],dtype=torch.float32),self.data[idx,1]


train,test=train_test_split(X2,test_size=.3,random_state=28)
train_dataloader=DataLoader(ChessDataSet(train),batch_size=batch_size,shuffle=True)
test_dataloader=DataLoader(ChessDataSet(test),batch_size=batch_size,shuffle=True)

len(train_dataloader),len(test_dataloader)
    

(22, 10)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model1=nn.Sequential(
    nn.BatchNorm2d(12),
    nn.Conv2d(12,32,3),
    nn.BatchNorm2d(32),
    nn.ReLU(),

    nn.Conv2d(32,64,3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Dropout2d(.2),

    nn.Conv2d(64,64,3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Dropout2d(.2),
    
    nn.Flatten(),

    nn.LazyLinear(64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout1d(.2),
             
    nn.Linear(64,32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout1d(.2),
             
    nn.Linear(32,1),
    nn.Tanh(),

    ).to(device)

train_losses,test_losses=[],[]

Using cuda device


In [ ]:
#define your loss function and optimizer (choice of optimizer is a hyperparameter)
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-3,weight_decay=0)

In [ ]:
# function that runs training loop
# specifiy epochs to control how long training runs
# verbose specifies how often we print model error logs 
#   verbose=1: every epoch (default) 
#   verbose=2: every other epoch
#   verbose=5: every 5th epoch 

# The function saves the weights of the version of the model that has the best
# validation loss.


def training_loop(model, train_loader, test_loader, loss_fn, optimizer, epochs,model_num,verbose=1):

    fp=f'/content/drive/MyDrive/chess/model{model_num}'
    saved_model=torch.load(fp)
    best_test_loss=saved_model['error']


    train_size,num_train_batches = len(train_loader.dataset),len(train_loader)
    test_size,num_test_batches=len(test_loader.dataset),len(test_loader)

    
    for epoch in tqdm(range(epochs)):
      print("________________________________________________________________")
      print()
      model.train()
      train_loss=0
      #correct=0
      for (X, y) in tqdm(train_loader):
          X, y = X.to(device), y.to(device)

          # Compute prediction error
          pred = torch.squeeze(model(X))
          loss = loss_fn(pred, y.float())

          train_loss+=loss.item()
          #correct+=((pred>.5)==(y==1)).sum().item()

          # Backpropagation
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      
      train_loss/=num_train_batches
      #train_acc=correct/train_size

      # Compute test error
      model.eval()
      test_loss=0
      #correct = 0
      with torch.no_grad():
        for X, y in test_loader:
          X, y = X.to(device), y.to(device)
          pred = torch.squeeze(model(X))
          test_loss += loss_fn(pred, y.float()).item()
          #correct += ((pred>.5)==(y==1)).sum().item()
      test_loss /= num_test_batches
      #test_acc = correct/test_size

      if test_loss<best_test_loss:
        best_test_loss=test_loss
        saved_model={
            'error':test_loss,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()
            }
        torch.save(saved_model,fp)


      train_losses.append(train_loss)
      test_losses.append(test_loss)
      if epoch%verbose==0 or epoch==epochs-1:
        print(f"Epoch {epoch}:")
        print(f"Train Loss:{train_loss}")# | Train Accuracy:{train_acc}")
        print(f"Test Loss:{test_loss}")# | Test Accuracy:{test_acc}")
    
    


      

      

In [ ]:
#first attempt
training_loop(model1,train_dataloader,test_dataloader,loss_fn,optimizer,50,1)


In [ ]:
# Now we define our own network architecture
# This model will involve skip connections and transposed convolutional layers

class Block(nn.Module):

  def __init__(self, num_channels):
    super().__init__()
    self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3)
    self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3)
    self.conv3 = nn.LazyConv2d(num_channels, kernel_size=3)

    self.deconv1=nn.LazyConvTranspose2d(num_channels,3)
    self.deconv2=nn.LazyConvTranspose2d(num_channels,3)
    self.deconv3=nn.LazyConvTranspose2d(num_channels,3)

    self.input_conv = nn.LazyConv2d(num_channels, kernel_size=1)
 
    self.bn1 = nn.LazyBatchNorm2d()
    self.bn2 = nn.LazyBatchNorm2d()
    self.bn3 = nn.LazyBatchNorm2d()
    self.bn4 = nn.LazyBatchNorm2d()
    self.bn5 = nn.LazyBatchNorm2d()
    self.bn6 = nn.LazyBatchNorm2d()

  def forward(self, X):
    X1 = self.bn1(self.conv1(X)) #6
    X2 = self.bn2(self.conv2(F.relu(X1))) #4
    X3 = self.bn3(self.conv3(F.relu(X2))) #2

    X4 = F.relu(self.bn4(self.deconv1(X3))+X2) #4
    X5 = F.relu(self.bn5(self.deconv2(X4))+X1) #6
    X6 = self.bn6(self.deconv3(X5)) #8


    X = self.input_conv(X)
    X6 += X
    return F.relu(X6)

class CustomNet(nn.Module):
  def __init__(self, channels):
    super().__init__()

    self.conv_net=nn.Sequential(*[Block(channel) for channel in channels])

    self.linear_classifier=nn.Sequential(
      nn.Flatten(),

      nn.LazyLinear(64),
      nn.BatchNorm1d(64),
      nn.ReLU(),
      #nn.Dropout1d(.2),
              
      nn.Linear(64,32),
      nn.BatchNorm1d(32),
      nn.ReLU(),
      #nn.Dropout1d(.2),
    
      nn.Linear(32,1),
      nn.Tanh(),
    )

  def forward(self,X):
    return self.linear_classifier(self.conv_net(X))





In [ ]:
model2=CustomNet([32,32,64,64])
model2=model2.to(device)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3,weight_decay=0)


# loading in previously saved model and optimizer
if Path("/content/drive/MyDrive/chess/model2").is_file():
  saved_model=torch.load("/content/drive/MyDrive/chess/model2",map_location=device)
  model2.load_state_dict(saved_model['model_state_dict'])
  optimizer2.load_state_dict(saved_model['optimizer_state_dict'])

loss_fn2 = nn.L1Loss()
train_losses,test_losses=[],[]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
training_loop(model2,train_dataloader,test_dataloader,loss_fn2,optimizer2,30,2)

In [ ]:
# Validation Loss
saved_model['error']

0.034205143217474324

In [ ]:
model3=CustomNet([32,32,64,64,128,128])
model3=model3.to(device)
optimizer3 = torch.optim.Adam(model3.parameters(), lr=1e-3,weight_decay=10)

if Path("/content/drive/MyDrive/chess/model3").is_file():
  saved_model=torch.load("/content/drive/MyDrive/chess/model3",map_location=torch.device('gpu'))
  model3.load_state_dict(saved_model['model_state_dict'])
  optimizer3.load_state_dict(saved_model['optimizer_state_dict'])

loss_fn3 = nn.L1Loss()
train_losses,test_losses=[],[]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
training_loop(model3,train_dataloader,test_dataloader,loss_fn3,optimizer3,5,3)

  0%|          | 0/5 [00:00<?, ?it/s]

________________________________________________________________



  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 0:
Train Loss:0.007321287073533644
Test Loss:0.026993631571531295
________________________________________________________________



  0%|          | 0/22 [00:00<?, ?it/s]

...

Epoch 2:
Train Loss:0.00756699638441205
Test Loss:0.0280596524477005
________________________________________________________________



  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 3:
Train Loss:0.007391090017997406
Test Loss:0.027882414497435094
________________________________________________________________



  0%|          | 0/22 [00:00<?, ?it/s]

Epoch 4:
Train Loss:0.007357864026826891
Test Loss:0.02784173283725977


In [ ]:
# Validation Error
# An L1 Loss of 0.027 is pretty decent considering the value ranges from [-1,1]
saved_model['error']

0.026993631571531295

In [ ]:
# Saving model as an onnx file
model3.eval()
torch.onnx.export(model3,torch.zeros(1,12,8,8),"/content/drive/MyDrive/chess/model3.onnx")